In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import dmc
from process import *

In [16]:
evaluation_sets = ['rawMirrored', 'rawLinearSample']
data = processed_data()

In [17]:
train, test = split_data_by_id(data, 'rawMirrored')

In [18]:
len(train), len(test)

(947782, 236498)

In [37]:
ensemble = dmc.ensemble.Ensemble(train[:10000], test[:10000])

In [38]:
for k in ensemble.splits:
    print(len(ensemble.splits[k][1]), k)

11 known_articleID-known_customerID-known_voucherID-known_productGroup
104 known_articleID-unknown_customerID-unknown_voucherID-known_productGroup
322 known_articleID-unknown_customerID-known_voucherID-known_productGroup
182 unknown_articleID-known_customerID-known_voucherID-known_productGroup
7004 unknown_articleID-unknown_customerID-known_voucherID-known_productGroup
2311 unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup
63 unknown_articleID-known_customerID-unknown_voucherID-known_productGroup
3 known_articleID-known_customerID-unknown_voucherID-known_productGroup


In [39]:
import csv
import pandas as pd

dropped_fts = ['t_singleItemPrice', 't_isTypePants', 't_isTypeBelt', 't_isTypeTop'
    , 't_isWeekend_A', 't_isOneSize_A', 't_voucher_is10PercentVoucher'
    , 't_voucher_is15PercentVoucher', 'customerAvgUnisize','voucherID']
file_name = 'ArticleProductgroupCustomer.csv'
with open('/Users/Fabian/Downloads/documents-export-2016-05-11/Sets_SVM_' +
                  file_name) as csv_file:
    for index, row in enumerate(csv.reader(csv_file, delimiter=',')):
        if index == 0:

            series_a = pd.Series(row[1:])
        else:
            series_b = pd.Series(row[1:])
feature_table = pd.DataFrame({'feature': series_a, "value": series_b})
unimp_fts = feature_table[feature_table['value'] == '0.0']['feature'].tolist()
for drop in dropped_fts:
    unimp_fts.remove(drop)
print(unimp_fts)

['colorCode', 'sizeCode', 'quantity', 'price', 'rrp', 'voucherAmount', 'deviceID', 'paymentMethod', 't_wsv', 't_ssv', 't_singleItemPrice_diff_rrp', 't_sizeCodeNumerized', 't_unisize', 't_paymentWithFee_A', 't_customer_boughtArticleCount', 't_customer_orderCount', 't_customer_voucherCount', 't_customer_avgUnisize', 't_order_boughtArticleTypeCount', 't_order_boughtArticleCount', 't_order_duplicateCount', 't_order_sameArticlesCount', 't_order_totalPrice', 't_order_meanPrice', 't_order_cheapestPrice', 't_order_mostExpensivePrice', 't_order_totalPrice_diff_voucherAmount', 't_order_priceStd_A', 't_order_hasVoucher_A', 't_order_sameArticlesCount_DiffSize', 't_order_sameArticlesCount_DiffColor', 't_article_priceChangeSTD_A', 't_order_article_sameArticlesCount', 't_order_article_sameArticlesCount_DiffColor', 't_voucher_isGiftVoucher', 't_voucher_isValueVoucher', 't_unisizeOffset', 'totalSavings', 'relativeSavings', 'orderYear', 'orderDay', 'orderDayOfYear', 'orderWeek', 'orderQuarter', 'orderTo

In [40]:
ensemble.transform(binary_target=True, scalers=[dmc.transformation.normalize_raw_features]*10, 
                   ignore_features=[unimp_fts] * 10)

/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [41]:
ensemble.classify([SVM] * len(ensemble.splits))

known_articleID-known_customerID-known_voucherID-known_productGroup : size  11  prec  0.7272727272727273
--------------------------------------
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup : size  104  prec  0.375
--------------------------------------
known_articleID-unknown_customerID-known_voucherID-known_productGroup : size  322  prec  0.5248447204968945
--------------------------------------
unknown_articleID-known_customerID-known_voucherID-known_productGroup : size  182  prec  0.5494505494505495
--------------------------------------
unknown_articleID-unknown_customerID-known_voucherID-known_productGroup : size  7004  prec  0.6344945745288406
--------------------------------------
unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup : size  2311  prec  0.6205106014712245
--------------------------------------
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup : size  63  prec  0.6984126984126984
---------------

In [35]:
len(ensemble.splits)

8